In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time
from requests.exceptions import ConnectionError

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [3]:
base = "C:/Users/valen/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


C:\Users\valen\AppData\Local\Temp\ipykernel_1816\3525900741.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [7]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list2 = []
risk_list2 = []

In [8]:
block_size = len(df_1) // 100
num_blocks = len(df_1) // 2035
num_blocks
2 * block_size
print(0 * block_size, 1 * block_size)
print(len(slugs))

0 2035
203510


In [ ]:
# instantiate a new Session 
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
           'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
          }
# GET request to kickstarter to get some token needed for later 
# r = s.get("https://www.kickstarter.com", headers=headers)
# BeautifulSoup parses the html returned as a response from the GET request 
# soup = BeautifulSoup(r.text, 'html.parser')
# xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
block_number = 0

headers = {
            'content-type': 'application/json',
            'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
            'user-agent': 'Mozilla/5.0 (Linux; U; Android 2.3.4; en-us; Kindle Fire Build/GINGERBREAD) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
            'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2'
          }

agents = [
         "Mozilla/5.0 (Symbian/3; Series60/5.2 NokiaN8-00/012.002; Profile/MIDP-2.1 Configuration/CLDC-1.1 ) AppleWebKit/533.4 (KHTML, like Gecko) NokiaBrowser/7.3.0 Mobile Safari/533.4 3gpp-gba",
         "Mozilla/5.0 (Linux; Android 5.0; Lenovo S60-a Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 YaBrowser/17.3.1.383.00 Mobile Safari/E7FBAF",
         "Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 244.0.0.12.112 (iPhone12,1; iOS 15_5; en_US; en-US; scale=2.00; 828x1792; 383361019)",
         "Mozilla/5.0 (Linux; U; Android 2.2.1; en-US; GT-S5830 Build/FROYO) AppleWebKit/534.31 (KHTML, like Gecko) UCBrowser/9.0.1.275 U3/0.8.0 Mobile Safari/534.31",
         "Mozilla/5.0 (Linux; Android 11; Redmi Note 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36"
         ]

# pause for 5 seconds every 5 iterations
for i in range(81404, 122106):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(30)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    connect_try = 0
    not_found = True
    while connect_try < 30 and not_found:       
        try:
            r = s.post("https://www.kickstarter.com/graph",
                headers = {
                  'content-type': 'application/json',
                  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
                  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
                  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
                },
                json = {
                    "operationName":"Campaign",
                    "variables":{
                        "slug": slug
                    },
                    "query": query
                })
            not_found = False
        except ConnectionError:
            print("Connection Error")
            time.sleep(5)
            connect_try += 1
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list2.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list2.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list2.append("")
        risk_list2.append("")
        continue

--------735729376/goldstein------
-------STORY--------
Social Credit System? Darunter können sich leider die wenigsten etwas vorstellen. Doch was genauso gut aus einem dystopischen Roman stammen könnte, ist in China leider bereits Realität. Der Staat ermittelt an Hand deines Verhaltens einen Punktestand. Dieser verändert sich mit deinem Handeln zum positiven oder negativen und entscheidet dabei maßgebend über dein Leben. Ob man ausreisen, öffentliche Verkehrsmittel nutzen oder zur Universität gehen kann, hängt ganz vom eigenen Credit Score ab. Dabei werden nicht nur moralisch oder gesetzlich falsche Taten als ausschlaggebend für die Verschlechterung des Scores aufgeführt, sondern auch der Konsum von zu vielen Videospielen o.ä., da dies als nicht für die Gesellschaft produktives Verhalten angesehen wird. 
Auch andere politische Meinungen werden diskreditiert und sanktioniert. Private Chats werden analysiert und auch der Kontakt mit Personen, die einen niedrigeren Punktestand haben, wird

-------STORY--------
Why do we need kickstarter you ask? I allocated the maximum legal withdrawal from my pension fund and am "all in" but have come up short. The moving target regarding upgrades to the space with both the City and Province has been, although enlightening, costly beyond my business plan. 
1958 is a project that has been in the planning stages for almost half a century. It is a restaurant offering always fresh, always local and sustainable food for the entire community. The restaurant is located in West Broadway, on the fringe of Wolseley and is part of a diverse, eclectic neighborhood where homes valued at over half a million dollars and gated communities are situated in close proximity to rooming houses and apartment buildings providing housing for those less fortunate. 
This business model is based on my belief system that simplicity is elegant and less is definitely more. Our offerings will be based on recipes that have stood strong for generations and are based on 

-------STORY--------
THE WILD HARE - A New Type of Restaurant for Arkadelphia Our 
goal is to create unforgettable food with fresh, local ingredients and 
provide a welcoming place for meeting friends, enjoying a meal with family, or treating that special someone. 
OUR VISION - More than just a bakery and a cafeWe love food, and we want to share that love with Arkadelphia. We believe even the most familiar foods can be made into luxurious treats with fresh, local ingredients and a bit of care. From muffins to grilled cheese to pork tenderloin, we promise to wow you. Plus we'll prepare mouthwatering dishes you won't find anywhere else in town. In addition to a full bakery complete with fresh breads, cakes, and pastries, The Wild Hare will serve breakfast, lunch, and dinner six days a week in a roomy building downtown. A covered outdoor patio will add to the downtown's growing nightlife, and an inviting indoor dining room will also serve as a showcase for local artists. We'll have Arkade

-------STORY--------
At the beginning of 2021, some new energy began to gather around a fresh set of hymns...
New arrangements and imaginative instrumentation took form...friends were gathered to sing and play...
And I couldn’t be more excited about what’s come of the progress so far.  I do need your help to finish the project, so this page is dedicated to communicating what's happening with it and opening up opportunities to be a part of it!
I'll be communicating a bunch via updates here on Kickstarter, but a whole lot more on my Instagram - Q&As, live streams, etc on all sorts of topics - gear, production, songs, etc.  So hop on over there to find me after you pledge: @aaronstrumpel.
_____________________________________________
A few fun details to not miss:
_____________________________________________
Current Track Listing:
_____________________________________________
A More Comprehensive Story:
After years of hearing people talk about it, I’ve been finally taking the time to rea

-------STORY--------
This is a story about mistrust. 5 agents have been tasked with observing and testing the subject of an experiment (that may or may not have been successful) for one year. With none of them allowed to leave the facility until their task is done, tensions rise as the group becomes evermore distrustful of each other, and the subject. 
The bulk of the funds from this campaign will go towards equipment; mainly cameras, microphones, and lights. The rest will go towards paying the actors. Since this is a movie with a small cast, we won't need to worry about having a lot of money to pay a lot of actors. We will use locations we have at our disposal, and props and costumes will come from thrift stores, so we won't need much for those either. This project will be minimal, with as little spending as possible, due to the fact that a smaller budget is much easier to obtain than a big budget.
This is a story I, personally, am passionate about. I love film, I want to make movies,

...

--------494538512/2013-holiday-3-cd-music-mix------
-------STORY--------
As you know, for 24 years, I've been sharing music with friends and family in the form of THREE CDs EVERY year.  These CDs include hand-picked music from around the world and often follow a theme, an idea, an artist, an era. The CDs are professionally replicated and include custom on-disc labeling and informative and entertaining disc inserts. Every year I spend months listening to music, making selections, arranging the selections, creating labels and inserts, negotiating with the replication vendor, assembling the jewel cases with inserts and gift wrapping each set. Then every set goes in a padded envelope, and I buy postage for each. Then, like musical magic, it arrives in your mailbox. 
This has been a tough year for me and I'd love your help in getting the music out the door for this, the 25th year. 
Components of my cost for the project include: purchasing music, buying blank CDRs to create multiple iteratio

-------STORY--------
Hello all, welcome to my Kickstarter page and a glimpse into what I'm looking to do and why I want to do this. First, I would like to say that I'm the last person in the world to ask for help, but the first person to try and lend a hand when I can. Recently I was jammed up and facing a roadblock that I just could not get around. Luckily my husband knew that I needed help  and made a call.  Thankfully I was able to receive the professional counseling I needed to put things back in perspective, and gave me a new outlook on my future.  During that time I reflected back on my 25+ years in emergency services, my life, my friends and family, what is important to me and what makes me happy, and I made a decision; I need to redirect my focus in a new venture!
The public safety field is unforgiving and unyielding.  I became an EMT when I was 22 years old, started dispatching shortly thereafter as well, and just 6 short years ago, at the age of 41 (what was I thinking) went 

-------STORY--------
Welcome.

 
This is a beginning of a very exciting time in my life where I will take all that I have learnt over the years and apply them to myself.

 
The idea of this project came into my mind after falling inside of a hole. I was lost and felt I needed to figure some things out. I was thinking a lot. Thinking about life and thinking about my future in the business. Then there it was. I had a vision.

 
The project itself is an opportunity for me as a chef to take myself to another level in my career. I always had a lot of creativity and now its time for me to show you. I will be traveling around the world to meet new experiences and to cook food in various restaurants where my menus will be on for people to enjoy.

 
My philosophy about food is simple. It has to be natural. Whatever produce I am using I want it to be the right time to use it and at the right place. I believe in sustainability and using all parts to minimize wastage. Mostly I prefer that food is 

-------STORY--------
Have you ever walked into a coffee & donut shop for there to be a whole 5 stale donuts left at 11am? I definitely have. It was many encounters as such that caused me to set out on a new start-up! A start-up with freshly roasted coffee daily, as well as daily cooked donuts & bagels all day long with no run outs! Our goal is not only freshness but customer satisfaction. We are from a town where the local bakery closes at 2 and the dunkin' donuts is either packed full or nothing left, and i am here to change that! 
-------RISKS--------
Our timeline for this project is to get started within the upcoming months ahead. The money raised will be used for kitchen equipment, as well as slightly remodeling parts of the leased building at our shaw's plaza location! Funds will also be used to obtain our new custom cups as well as our new coffee packaging!
--------johnchurchville/saha-booker-churchville-cd-production------
-------STORY--------
Hello and thanks for checking out o

-------STORY--------
Tropic Zine is a forum for critical engagement with contemporary culture that seeks connections between Hawaiʻi and the tropical diaspora worldwide. Based in Honolulu, we are a print publication, digital platform, and creative collective. Our editorial content includes visual art, essays, poetry, interviews, photography, new media, and research-driven explorations; we are a collation of new cultural material that defies any singular narrative. We embrace queer modes of thinking, discontinuity, and hybridity, while prioritizing under-represented, emerging, and under-recognized artists.
In March 2018, Tropic Zine raised enough funds to produce our first issue thanks to the generous support of our friends, families, strangers, and extended community. We released Issue 1 in September, started a non-profit publishing imprint, Tropic Editions, and received our 501(c)(3) tax exemption from the IRS. 
We have continued to evolve and expand our reach and our mission through 

## Latest Iteration Ran Till: 29029, Local copy latest: 0_29029

In [433]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
story_file_name = "data/81404_122106_story" #CHANGE THIS
risk_file_name = "data/81404_122106_risk" #CHANGE THIS 

story_file_name += ".json"
risk_file_name += ".json"

with open(story_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(story_list2, f, indent=2) 

with open(risk_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(risk_list2, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

In [431]:
print("risk:", len(risk_list2),"\n", "story:", len(story_list2))

risk: 29030 
 story: 29030


In [432]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[len(story_list2)-1]['blurb'])

print("LATEST STORY IN LIST", "\n")
print(story_list2[-1])

print("LATEST RISK IN LIST", "\n")
print(risk_list2[-1])

LATEST CALL DF ROW 

Biggest Heart Felt Cultural Cross over indie Film hit of the year! #SubwaySon
LATEST STORY IN LIST 

Aden Mc Ready, a bastard child of a drug and alcohol addled mom and an unidentified dad is reduced to singing in the subway for handouts. He is jolted into the final realization that the only way to predict ones future is to create it through faith, blood, sweat and tears. After numerous struggles, recoveries, and relapses he accidentally meets the man who is his dad, a man who once had it all but would give it all back to have been able to just have been present for his only son. Aden's attribute success to the emotions he went through as a neglected child. Now, when surrounded by the love and caring that has eluded him all his life, Aden starts his own family armed with the desperate plea of a dying man (his father) never to neglect his children. Adam Ratcliffe will be premiering in two movies scheduled to release this year!  Brotherly love starring Keke Palmer, C

In [416]:
# story_list2 = story_list2[:-130]
# risk_list2 = risk_list2[:-130]
# story_list2 = story_list2[:-5]
# risk_list2 = risk_list2[:-5]
# headers = {
#             'content-type': 'application/json',
#             'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
#             'user-agent': 'Mozilla/5.0 (Linux; U; Android 2.3.4; en-us; Kindle Fire Build/GINGERBREAD) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
#             'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2'
#           }
# agents = [
#          "Mozilla/5.0 (Symbian/3; Series60/5.2 NokiaN8-00/012.002; Profile/MIDP-2.1 Configuration/CLDC-1.1 ) AppleWebKit/533.4 (KHTML, like Gecko) NokiaBrowser/7.3.0 Mobile Safari/533.4 3gpp-gba",
#          "Mozilla/5.0 (Linux; Android 5.0; Lenovo S60-a Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 YaBrowser/17.3.1.383.00 Mobile Safari/E7FBAF",
#          "Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 244.0.0.12.112 (iPhone12,1; iOS 15_5; en_US; en-US; scale=2.00; 828x1792; 383361019)",
#          "Mozilla/5.0 (Linux; U; Android 2.2.1; en-US; GT-S5830 Build/FROYO) AppleWebKit/534.31 (KHTML, like Gecko) UCBrowser/9.0.1.275 U3/0.8.0 Mobile Safari/534.31",
#          "Mozilla/5.0 (Linux; Android 11; Redmi Note 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36"
#          ]
# headers['user-agent'] = agents[0]

# headers
# story_list[-130]
#story_list[:-130][-1]

# with open("./data/0_4289_story.json", 'r') as f:
#     story_list2 = json.load(f)
# with open("./data/0_4289_risk.json", 'r') as f1:
#     risk_list2 = json.load(f1)

In [165]:
s = requests.Session()

In [421]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

200
This is my fourth solo Kickstarter campaign, and I delivered all three of my previous games in a timely manner. 

I have a complete draft of the text, with the exception of Appendix 1. I have a team of editors lined up who will give me feedback so that I can revise, expand, and polish it as needed. The most time-consuming tasks will be completing the artwork and the layout. I've done this before, with the previous Akashic Titan zine and with my game Project 8Ball. Depending on the stretch goals I will be depending on someone else for illustrations and additional text, but I have worked with Josh before and I am confident he will hit his deadlines. 

I believe I will be able to finish the book by June, but just to be cautious I have set the estimated delivery month as July. 

Barring any unforeseen catastrophes, I should have no problem printing and shipping the zine by July.


In [420]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": slugs[4290]
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
  'user-agent': 'Mozilla/5.0 (Linux; U; Android 2.3.4; en-us; Kindle Fire Build/GINGERBREAD) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
  'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2h08X3WX0lQSBSeatHjTEHV%2FgAjhqitXq1xvrNIY7MvvonoTMQat4mSbJKrOzGi74PUMwlhlgHz1oDCWy%2BqAiY%2BIQp9ft4c%2BK2ta8%3D--Rds4wbR2FagzmGoy--ZV4JbIYN8J%2FdS8uWWa5EQw%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [255]:
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          }
# GET request to kickstarter to get some token needed for later 
r = s.get("https://www.kickstarter.com", headers=headers)
#BeautifulSoup parses the html returned as a response from the GET request 
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

In [ ]:
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'

In [237]:
print("hello")
time.sleep(3)
print("goodnight")

hello
goodnight
